# Fourier transform

In [31]:
using Revise
using Plots
using LinearAlgebra
using SparseIR
import SparseIR: valueim
using MultiScales

newaxis = [CartesianIndex()]

1-element Vector{CartesianIndex{0}}:
 CartesianIndex()

In [32]:
BLAS.set_num_threads(16)

In [33]:
println(Threads.nthreads())

1


In [34]:
using ITensors

println(ITensors.blas_get_num_threads())

16


In [35]:
nbit = 4
N = 2^nbit

sites = siteinds("Qubit", nbit)
M = MultiScales._qft(sites)

MPO
[1] ((dim=2|id=776|"Qubit,Site,n=1"), (dim=2|id=776|"Qubit,Site,n=1")', (dim=2|id=399|"Link,n=1"))
[2] ((dim=2|id=399|"Link,n=1"), (dim=2|id=80|"Qubit,Site,n=2"), (dim=2|id=80|"Qubit,Site,n=2")', (dim=4|id=39|"Link,n=2"))
[3] ((dim=4|id=39|"Link,n=2"), (dim=2|id=683|"Qubit,Site,n=3"), (dim=2|id=683|"Qubit,Site,n=3")', (dim=2|id=643|"Link,n=3"))
[4] ((dim=2|id=643|"Link,n=3"), (dim=2|id=65|"Qubit,Site,n=4"), (dim=2|id=65|"Qubit,Site,n=4")')


In [36]:
# Return the bit of an integer `i` at the position `pos` (`pos=1` is the least significant digit).
bitat(i, pos) = ((i & 1<<(pos-1))>>(pos-1))
@assert bitat(2, 1) == 0
@assert bitat(2, 2) == 1

In [37]:
# Input function `f(x)` is 1 only at xin otherwise 0.
xin = 3 # 1-start
@assert xin <= N
tmp = collect(string(bitat(xin-1, pos)) for pos in nbit:-1:1)
mpsf = MPS(sites, tmp)
@show tmp

tmp = ["0", "0", "1", "0"]


4-element Vector{String}:
 "0"
 "0"
 "1"
 "0"

In [38]:
#reduce(*, mpsf)[sites[1]=>1, sites[2]=>2]

In [39]:
# Physical indices from lef to right: t_Q, ..., t_1
mpsg = reduce(*, noprime(contract(M, mpsf)))

# Values of output function
outfunc = vec(Array(mpsg, sites))

16-element Vector{ComplexF64}:
     0.25000000000000017 + 6.400983057067571e-17im
     0.17677669529663692 + 0.17677669529663698im
 -2.5389538253082465e-17 + 0.25000000000000017im
    -0.17677669529663714 + 0.1767766952966369im
     -0.2500000000000004 - 2.056573309268645e-16im
    -0.17677669529663756 - 0.17677669529663703im
   7.097585253283989e-17 - 0.2500000000000003im
      0.1767766952966369 - 0.17677669529663725im
      0.2500000000000001 - 6.172552310525105e-17im
     0.17677669529663698 + 0.17677669529663687im
  1.3104908732216735e-16 + 0.2500000000000001im
    -0.17677669529663703 + 0.176776695296637im
     -0.2500000000000003 + 3.5056919146304366e-16im
    -0.17677669529663723 - 0.1767766952966368im
 -4.9065389333867966e-17 - 0.2500000000000002im
     0.17677669529663678 - 0.1767766952966373im

In [40]:
sign = 1
@assert outfunc ≈ [exp(sign * im * 2π * (t-1) * (xin-1)/N)/sqrt(N) for t in 1:N]